In [1]:
import time
import networkx
import numpy as np
import scipy
from scipy.optimize import linear_sum_assignment
from scipy.spatial import Delaunay
from scipy.sparse import csr_matrix

In [2]:
###################################
########    Step size      ########
###################################

def tr_dot(a, b):

    # 快速计算 np.trace(a*b)

    trace = np.sum(np.multiply(a, b.T))  # 点乘
    # trace=np.sum(inner1d(a, b.T))#内积
    # trace=np.einsum('ij,ji->', a, b)
    return trace

def compute_alpha(M, delta_f, D, adjacency1, adjacency2, K, lamb):
    tr_matrix1 = tr_dot(M.T, delta_f)
    tr_matrix2 = tr_dot(delta_f, D.T)
    tr_matrix3 = tr_matrix2
    tr_matrix4 = tr_dot(D.T * adjacency1, D * adjacency2)
    if np.max(K) == 0:
        tr_matrix5 = 0
        tr_matrix6 = 0
    else:

        tr_matrix5 = tr_dot(D.T, K)
        tr_matrix6 = tr_dot(M.T, K)

    tr_matrix1m2m3 = tr_matrix1 - tr_matrix2 - tr_matrix3
    alpha_a = tr_matrix1m2m3 + tr_matrix4
    alpha_b = -tr_matrix1m2m3 - tr_matrix1 + lamb * tr_matrix5 - lamb * tr_matrix6
    if alpha_b==0:
        alpha_op=1
    else:
        alpha_op = -alpha_b / (2 * alpha_a)
        
    return alpha_op

In [49]:
###################################
########    CSGO module     #######
###################################

def sinkhorn_vec(M, num_iters=1000,tol=0.05):
    
    M = np.array(M)
    n, m = M.shape

    # Initialize the scaling factors u and v
    u, v = np.ones(n), np.ones(m)
    r, c = np.ones(n), np.ones(m)
    
    matrix_old = M
    # Run Sinkhorn iterations
    for i in range(num_iters):
        u_new = 1 / np.matmul(M, v)  #Sum of row 
        v_new = 1 / np.matmul(M.T, u_new) #Sum of clo

        if i % 20 ==1: # Stopping test
            res_diff =  np.linalg.norm(u_new/max(u_new) - u/max(u), 1) 
            + np.linalg.norm(v_new/max(v_new) - v/max(v), 1)
            if res_diff<tol:
                print('sinkhorn converges at : '+ str(i))
                u, v = u_new, v_new
                break

        u, v = u_new, v_new 

    # Compute the optimal transport plan P
    scale_matrix = np.outer(u_new,v_new)
    P_eps = np.multiply(scale_matrix, M)
    
    P_eps = np.mat(P_eps)
    return P_eps


def scal_softassign(M, gamma=1, num_iter=1000,tole=0.001, stable=0):

    # Exponentiate the cost matrix M to create K
    n, m = M.shape
    M = M/ M.max()
    if stable: M= M -1
    beta = np.log((n + m) / 2) * gamma   #
    K = np.exp(M * beta)
    
    P_eps = sinkhorn_vec(K, num_iters=num_iter,tol=tole)

    return P_eps


def hugarian(matrix):
    n, m = matrix.shape
    P = np.mat(np.zeros((n, m)))
    row_ind, col_ind = linear_sum_assignment(-matrix)
    P[row_ind, col_ind] = 1
    return P


def graphmatch_CSGO(
    adjacency1, adjacency2, K=0, erro=0.1, a=1, lamb=1, gamma=10, adaptive=0,tole=1e-2,num_iter=1000,stable=0
):
    print("##################### CSGO #############################")
    starttime = time.perf_counter()

    n, _ = adjacency1.shape
    m, _ = adjacency2.shape
    big_nm = max(n, m)
    X = np.mat(np.ones((n, m))) / (n * m)
    Y = np.mat(np.zeros((big_nm, big_nm)))
    adj1 = csr_matrix(adjacency1)
    adj2 = csr_matrix(adjacency2)

    gtime = 0
    for i in range(60):
        x = X
        if i==0:
            degree_adjacency1 = np.sum(adjacency1,axis=1)
            degree_adjacency2 = np.sum(adjacency2,axis=1)
            delta_edge = np.mat(np.outer(degree_adjacency1, degree_adjacency2))/ (n * m)
        else:
            start_time = time.time()
            delta_edge = adj1@X@adj2
            end_time = time.time()
            gtime = gtime+(end_time-start_time)
        Y[0:n, 0:m] = delta_edge + lamb * K
        Y = scal_softassign(Y, gamma, num_iter, tole,stable=stable)

        M = x
        D = Y[0:n, 0:m]

        if adaptive:
            alpha_op = compute_alpha(M, delta_edge, D, adjacency1, adjacency2, K, lamb)
        else:
            alpha_op = 1

        if alpha_op >= 0 and alpha_op < 1:
            X = (1 - alpha_op) * X + alpha_op * Y[0:n, 0:m]
            print("alpha is" + str(alpha_op))
        else:
            X = (1 - a) * X + a * Y[0:n, 0:m]

        err = abs(x / x.max() - X / X.max()).max()
        print(i, err)
        nIteration = i + 1
        if err < erro:
            print("converge")
            break
    P = hugarian(X)
    endtime = time.perf_counter()
    runtime = endtime - starttime
    print(gtime)
    return P, runtime, nIteration

In [41]:
def graphmatch_CSGO_ini_spa(
    adjacency1, adjacency2, K=0, erro=0.1, a=1, lamb=1, gamma=10, adaptive=0,tole=0.01,num_iter=1000,stable=0
):
    print("##################### CSGO #############################")
    starttime = time.perf_counter()

    n, _ = adjacency1.shape
    m, _ = adjacency2.shape
    big_nm = max(n, m)
    X = np.mat(np.ones((n, m))) / (n * m)
    Y = np.mat(np.zeros((big_nm, big_nm)))
    adj1 = csr_matrix(adjacency1)
    adj2 = csr_matrix(adjacency2)

    gtime = 0
    for i in range(60):
        x = X
        if i==0:
            degree_adjacency1 = np.sum(adjacency1,axis=1)
            degree_adjacency2 = np.sum(adjacency2,axis=1)
            delta_edge = np.mat(np.outer(degree_adjacency1, degree_adjacency2))/ (n * m)
        else:
            start_time = time.time()
            delta_edge = adj1@X@adj2
            end_time = time.time()
            gtime = gtime+(end_time-start_time)
        Y[0:n, 0:m] = delta_edge + lamb * K
        Y = scal_softassign(Y, gamma, num_iter, tole,stable=stable)

        M = x
        D = Y[0:n, 0:m]

        if adaptive:
            alpha_op = compute_alpha(M, delta_edge, D, adjacency1, adjacency2, K, lamb)
        else:
            alpha_op = 1

        if alpha_op >= 0 and alpha_op < 1:
            X = (1 - alpha_op) * X + alpha_op * Y[0:n, 0:m]
            print("alpha is" + str(alpha_op))
        else:
            X = (1 - a) * X + a * Y[0:n, 0:m]

        err = abs(x / x.max() - X / X.max()).max()
        print(i, err)
        nIteration = i + 1
        if err < erro:
            print("converge")
            break
    P = hugarian(X)
    endtime = time.perf_counter()
    runtime = endtime - starttime
    print(gtime)
    return P, runtime, nIteration

# Experiment： PPI

In [30]:
sorce_graph = networkx.read_leda("PPI/synthetic_nets_known_node_mapping/0Krogan_2007_high.gw")
nosiy5_graph = networkx.read_leda("PPI/synthetic_nets_known_node_mapping/low_confidence/0Krogan_2007_high+5e.gw")
nosiy15_graph = networkx.read_leda("PPI/synthetic_nets_known_node_mapping/low_confidence/0Krogan_2007_high+15e.gw")
nosiy25_graph = networkx.read_leda("PPI/synthetic_nets_known_node_mapping/low_confidence/0Krogan_2007_high+25e.gw")

adj_sorce = networkx.to_numpy_matrix(sorce_graph)
adj_noisy5 = networkx.to_numpy_matrix(nosiy5_graph)
adj_noisy15 = networkx.to_numpy_matrix(nosiy15_graph)
adj_noisy25 = networkx.to_numpy_matrix(nosiy25_graph)
nodes =1004

In [54]:
M, runtime, iter_num  = graphmatch_CSGO(adj_sorce, adj_noisy5,gamma =60,stable=1)
print("Run time: "+str(np.round(runtime,2))+"sec")
print("Node acc: "+str(np.round((sum(np.diag(M))/nodes)*100,2))+"%") 

##################### CSGO #############################
sinkhorn converges at : 141
0 1.0
sinkhorn converges at : 341
1 0.9781336989094666
sinkhorn converges at : 281
2 0.957331826518079
sinkhorn converges at : 201
3 0.9327478527017503
sinkhorn converges at : 221
4 0.6944889077070697
sinkhorn converges at : 201
5 0.7214800467719892
sinkhorn converges at : 201
6 0.7413081843559142
sinkhorn converges at : 181
7 0.728613095677342
sinkhorn converges at : 181
8 0.8762599600901462
sinkhorn converges at : 161
9 0.40193905145778996
sinkhorn converges at : 141
10 0.4294822123350936
sinkhorn converges at : 141
11 0.3399825231085223
sinkhorn converges at : 141
12 0.38699933408383846
sinkhorn converges at : 141
13 0.37064844959211013
sinkhorn converges at : 141
14 0.27484666434279503
sinkhorn converges at : 141
15 0.2892829987667736
sinkhorn converges at : 141
16 0.2690971484543248
sinkhorn converges at : 141
17 0.19946762437063104
sinkhorn converges at : 141
18 0.2208253265223693
sinkhorn conver

# Experiment: Facebook

In [21]:
G_face = networkx.read_gpickle("facebook/G_face.gpickle")  
G_face_noise_5 = networkx.read_gpickle("facebook/G_face_noise_5.gpickle")
G_face_noise_15 = networkx.read_gpickle("facebook/G_face_noise_15.gpickle")
G_face_noise_25 = networkx.read_gpickle("facebook/G_face_noise_25.gpickle")

adj = networkx.to_numpy_matrix(G_face)
adj_5 = networkx.to_numpy_matrix(G_face_noise_5)
adj_15 = networkx.to_numpy_matrix(G_face_noise_15)
adj_25 = networkx.to_numpy_matrix(G_face_noise_25)

In [56]:
nodes=4039
M, runtime, nIter =graphmatch_CSGO(adj, adj_25, adaptive=0,gamma =60,stable=1)
print("Run time: "+str(np.round(runtime,2))+"sec")
print("Node acc: "+str(np.round((sum(np.diag(M))/nodes)*100,2))+"%") 

##################### CSGO #############################
sinkhorn converges at : 81
0 1.0
sinkhorn converges at : 241
1 1.0
sinkhorn converges at : 521
2 0.9646127685671819
sinkhorn converges at : 421
3 0.99999999999956
sinkhorn converges at : 401
4 0.8310143232104267
sinkhorn converges at : 221
5 0.9403287074676424
sinkhorn converges at : 161
6 0.6792245184042067
sinkhorn converges at : 181
7 0.6736730313369764
sinkhorn converges at : 201
8 0.6830258658317601
sinkhorn converges at : 181
9 0.7265261608056721
sinkhorn converges at : 201
10 0.6511333123978195
sinkhorn converges at : 201
11 0.718812615334375
sinkhorn converges at : 201
12 0.5528168284387799
sinkhorn converges at : 201
13 0.46006279748494205
sinkhorn converges at : 201
14 0.39485714790118986
sinkhorn converges at : 201
15 0.1912511440317397
sinkhorn converges at : 201
16 0.13255559371816766
sinkhorn converges at : 201
17 0.08954825598399407
converge
23.68960928916931
Run time: 92.71sec
Node acc: 86.26%
